# Introduction
This notebook demonstrates the integration capabilities of roboquant with OANDA. OANDA is a broker with a focus on FOREX, but also offers CFD for sevral commodities. 

There are currently three types of integration available:

1. Use the historic data 
2. Use live data 
3. Use thetrading functionality (WIP)


By default the required OANDA classes are not imported, so we need to do that first. This will load the Feed and Broker classes for integrating with OANDA.

In [ ]:
%use roboquant
import org.roboquant.oanda.*
Welcome()

# Back Testing
The first example shows how to use historic data from OANDA in a back test. To use Alpaca you'll need to authenticate with a Key and Secret. There are several ways to pass them to the feed, either directly or through system environment or a property file:


<div class="alert alert-block alert-info">
Make sure you use the demo account since roboquant is still very much in Beta
</div>

In [ ]:
val feed = OANDAHistoricFeed()
feed.retrieveCandles("EUR_USD", "GBP_USD")
feed.assets.summary()

In [ ]:
PriceBarChart(feed, feed.assets.first())

In [ ]:
val strategy = EMACrossover.midTerm()
val roboquant = Roboquant(strategy, AccountSummary(), ProgressMetric())
roboquant.run(feed)

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val metric = roboquant.logger.getMetric("account.value")
MetricChart(metric)

In [ ]:
PriceBarChart(feed, feed.assets.last(), account.trades)

# Live Testing
Now we create an instance of the OANDALiveFeed and subscribe to three forex pairs. In order to keep it simple, in this example we subscribe to pairs denoted in USD so we can trade without having to define a currency converter. 

In [ ]:
val feed = OANDALiveFeed()
feed.subscribeOrderBook("EUR_USD", "GBP_USD", "AUD_USD")

We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. 
We'll run it for 5 minutes, but you can change this. But remember the EMA Crossover is only going to evaluate after at least three observations.

If the time that is displayed in the progress bar looks off by a few hours, please realize that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [ ]:
val strategy = EMACrossover.shortTerm()
val metric = PriceMetric(feed.assets.first())

val roboquant = Roboquant(strategy, AccountSummary(), metric)
val timeFrame = TimeFrame.nextMinutes(5)
roboquant.run(feed, timeFrame)

In [ ]:
val strategy = EMACrossover.midTerm()
val metric = PriceMetric(feed.assets.first())

val currencyConverter = FixedExchangeRates(Currency.EUR, Currency.USD to 0.9, Currency.GBP to 1.2)
val account = Account(Currency.EUR, currencyConverter = currencyConverter)
val broker = SimBroker(account = account)
val roboquant = Roboquant(strategy, AccountSummary(), metric, broker =  broker)

val timeFrame = TimeFrame.nextMinutes(5)
roboquant.run(feed, timeFrame)


## Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
Currency.USD.defaultFractionDigits = 4
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
PriceChart(metric, metric.asset, account.trades)

In [ ]:
val logger = roboquant.logger
logger.summary().print()

In [ ]:
logger.getMetricNames().summary()

## Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
val accountValue = logger.getMetric("account.value")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeChart(account.trades)

# Paper Trading
And just as it was the case in the previous section, most live feeds will only generate data during trading hours. So if you run these code cells outside trading hours, you won't see signals and orders being generated.

<div class="alert alert-block alert-warning">
The integration with the trading API is still very much work in progress, so only use this with a paper trading account
</div>

In [ ]:
val currencyConverter = FixedExchangeRates(Currency.EUR, 
                                          Currency.USD to 0.9, 
                                          Currency.GBP to 1.2)
val broker = OANDABroker(currencyConverter = currencyConverter)
val account = broker.account

In [ ]:
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val strategy = EMACrossover(3, 5)
val roboquant = Roboquant(strategy, AccountSummary(), ProgressMetric(), broker = broker)

The setup of the feed is exactly the same as before. There is no difference in the feed if you use it for live testing with the builtin Simulated Broker or paper- and live-trading. Alos this time we'll run it for 30 minutes.

In [ ]:
val feed = OANDALiveFeed()
val symbols = account.portfolio.assets.map { it.symbol }.toTypedArray()
feed.subscribeOrderBook(*symbols)

In [ ]:
val timeFrame = TimeFrame.nextMinutes(5)
roboquant.run(feed, timeFrame)

In [ ]:
account.portfolio.summary()